In [1]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from loader_util.preprocessing import ImageToArrayPreprocessor, \
    AspectAwarePreprocessor, \
    MeanSubtractionPreProcessor
from loader_util.datasets import SimpleDatasetLoader
from loader_util.nn.conv import FCHeadNet
##
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from imutils import paths
import numpy as np
import os

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
args = {
    'dataset':r"/home/mhasan3/Desktop/WorkFolder/cellImages3/"}

In [3]:
# construct the image generator
aug = ImageDataGenerator(rotation_range=30,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.2,
                         zoom_range=0.2, 
                         horizontal_flip=True,
                         vertical_flip=True,
                         fill_mode='nearest')

In [4]:
# grab the list of images that we'll be describing then extract the class 
# label names from the imagePaths
imagePaths = list(paths.list_images(args['dataset']))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

In [5]:
# initialise the image preprocessor
msp = MeanSubtractionPreProcessor()
aap = AspectAwarePreprocessor(224,224)
iap = ImageToArrayPreprocessor()

# load the dataset from disk then scale the raw pixels
sdl = SimpleDatasetLoader(preprocessors=[msp, aap, iap])
data, labels = sdl.load(imagePaths, verbose=500)
#data = data.astype('float') / 255.0

[INFO] processed 500/1728
[INFO] processed 1000/1728
[INFO] processed 1500/1728


In [6]:
# partition the data into training and test splits
trainx, testx, trainy, testy = train_test_split(data,
                                                labels,
                                                test_size=0.25,
                                                random_state=42)

# convert labels from integers into vectors
le = LabelBinarizer()
trainy = le.fit_transform(trainy)
testy = le.transform(testy)

In [7]:
le.classes_

array(['3T3', 'MG 63', 'hASC'], dtype='<U5')

In [30]:
# load the VGG16 network ensuring the head FC layer sets are left off
baseModel = VGG16(weights='imagenet', include_top=False, 
                  input_tensor=Input(shape=(224, 224, 3))) # type: Model

# initialise the new head of the network, a set of FC layers followed by 
# softmax
headModel = FCHeadNet.builld(baseModel, len(classNames), D=256) # type: Model

# place the head FC model on top of the base model - this will be the actual
# model to train
model = Model(inputs=baseModel.input, outputs=headModel)

(?, 7, 7, 512)


In [31]:
# loop over all the layers in the base model and freeze them
for layer in baseModel.layers:
    layer.trainable = False

In [32]:
# compile and optimise model
opt = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# train the head for few epochs
model.fit_generator(aug.flow(trainx, trainy, batch_size=32),
                    validation_data=(testx, testy),
                    epochs=25,
                    steps_per_epoch=len(trainx) // 32,
                    verbose=1)

Epoch 1/25
41/41 [==============================] - 16s 385ms/step - loss: 4.2349 - acc: 0.6296 - val_loss: 0.3787 - val_acc: 0.9306
Epoch 2/25
41/41 [==============================] - 15s 370ms/step - loss: 0.1865 - acc: 0.9483 - val_loss: 0.0373 - val_acc: 0.9907
Epoch 3/25
41/41 [==============================] - 15s 372ms/step - loss: 0.1517 - acc: 0.9552 - val_loss: 0.0321 - val_acc: 0.9907
Epoch 4/25
41/41 [==============================] - 15s 373ms/step - loss: 0.1189 - acc: 0.9622 - val_loss: 0.0263 - val_acc: 0.9931
Epoch 5/25
41/41 [==============================] - 15s 370ms/step - loss: 0.1188 - acc: 0.9745 - val_loss: 0.0232 - val_acc: 0.9977
Epoch 6/25
41/41 [==============================] - 15s 374ms/step - loss: 0.1300 - acc: 0.9699 - val_loss: 0.0108 - val_acc: 0.9954
Epoch 7/25
41/41 [==============================] - 15s 375ms/step - loss: 0.1110 - acc: 0.9815 - val_loss: 0.0312 - val_acc: 0.9977
Epoch 8/25
41/41 [==============================] - 15s 372ms/step - 

In [33]:
# evaluate the network
preds = model.predict(testx, batch_size=32)
print(classification_report(testy.argmax(axis=1),
                            preds.argmax(axis=1),
                            target_names=classNames))

              precision    recall  f1-score   support

         3T3       0.99      1.00      1.00       147
       MG 63       1.00      0.99      1.00       119
        hASC       1.00      1.00      1.00       166

    accuracy                           1.00       432
   macro avg       1.00      1.00      1.00       432
weighted avg       1.00      1.00      1.00       432



In [34]:
# now that FC layers have been trained, lets unfreeze the final set of CONV 
# layers and make them trainable
for layer in baseModel.layers[15:]:
    layer.trainable = True

In [35]:
# for changes to take effect we need to recompile model
opt = SGD(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.fit_generator(aug.flow(trainx, trainy, batch_size=32),
                    validation_data=(testx, testy),
                    epochs=100,
                    steps_per_epoch=len(trainx) // 32,
                    verbose=1)

Epoch 1/100
41/41 [==============================] - 17s 408ms/step - loss: 1.6632 - acc: 0.8565 - val_loss: 0.0204 - val_acc: 0.9931
Epoch 2/100
41/41 [==============================] - 16s 398ms/step - loss: 0.0270 - acc: 0.9923 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 3/100
41/41 [==============================] - 17s 403ms/step - loss: 0.0165 - acc: 0.9977 - val_loss: 0.0016 - val_acc: 1.0000
Epoch 4/100
41/41 [==============================] - 16s 399ms/step - loss: 0.0089 - acc: 0.9977 - val_loss: 2.8949e-04 - val_acc: 1.0000
Epoch 5/100
41/41 [==============================] - 16s 401ms/step - loss: 0.0043 - acc: 0.9985 - val_loss: 0.0051 - val_acc: 0.9977
Epoch 6/100
41/41 [==============================] - 17s 403ms/step - loss: 0.0036 - acc: 0.9985 - val_loss: 5.0772e-04 - val_acc: 1.0000
Epoch 7/100
41/41 [==============================] - 17s 405ms/step - loss: 0.0155 - acc: 0.9977 - val_loss: 0.0048 - val_acc: 0.9977
Epoch 8/100
41/41 [==============================] - 1

Epoch 37/100
41/41 [==============================] - 16s 401ms/step - loss: 0.0019 - acc: 0.9992 - val_loss: 5.9210e-07 - val_acc: 1.0000
Epoch 38/100
41/41 [==============================] - 16s 402ms/step - loss: 0.0036 - acc: 0.9985 - val_loss: 5.3368e-07 - val_acc: 1.0000
Epoch 39/100
41/41 [==============================] - 16s 399ms/step - loss: 0.0099 - acc: 0.9985 - val_loss: 2.9307e-04 - val_acc: 1.0000
Epoch 40/100
41/41 [==============================] - 17s 403ms/step - loss: 8.3541e-04 - acc: 1.0000 - val_loss: 8.9054e-07 - val_acc: 1.0000
Epoch 41/100
41/41 [==============================] - 17s 404ms/step - loss: 2.9541e-04 - acc: 1.0000 - val_loss: 5.4990e-07 - val_acc: 1.0000
Epoch 42/100
41/41 [==============================] - 16s 402ms/step - loss: 1.0410e-04 - acc: 1.0000 - val_loss: 3.3794e-07 - val_acc: 1.0000
Epoch 43/100
41/41 [==============================] - 16s 401ms/step - loss: 1.4789e-04 - acc: 1.0000 - val_loss: 2.4800e-07 - val_acc: 1.0000
Epoch 44/10

In [36]:
# evaluate the network again
preds = model.predict(testx, batch_size=32)
print(classification_report(testy.argmax(axis=1),
                            preds.argmax(axis=1),
                            target_names=classNames))

              precision    recall  f1-score   support

         3T3       1.00      1.00      1.00       147
       MG 63       1.00      1.00      1.00       119
        hASC       1.00      1.00      1.00       166

    accuracy                           1.00       432
   macro avg       1.00      1.00      1.00       432
weighted avg       1.00      1.00      1.00       432



In [38]:
# save serialised model
model.save('keras_finetune_acc1.00.pth')